In [1]:
import numpy as np
import pandas as pd
import jax
import jax.numpy as jnp

import plotly.express as px

from datetime import datetime
import time

/var/folders/ht/fh4bytd97pz3m_k1s82rf1vw0000gn/T/ipykernel_76805/1428000839.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
class ReactiveFF:
    def __init__(
        self,
        atom_list,
        xyz,
        box,
        gamma=1.0,
        seed=42,
    ):
        np.random.seed(seed)
        
        # particle params
        self.N = len(atom_list)
        self.box = jnp.array(box)
        self.inv_box = jnp.diag(1.0 / jnp.diag(self.box))

        self.X = jnp.asarray(xyz, dtype='float32')
        self.V = jnp.zeros_like(xyz)
        self.F = jnp.zeros_like(xyz)
        self.F_cube = np.zeros((N, N, 3), dtype='float32')
        # self.vir = None

        # sampling params
        self.kT = 1.0
        self.gamma = gamma

        # integraton params
        self.n_steps = None
        self.dt = None

        # minimisation params
        self.n_min = None

        # initial functions
        self._init_speeds()

    def _init_speeds(self):
        self.V = np.random.randn(self.N, 3) * np.sqrt(self.kT)
        self.V = jnp.asarray(self.V)
        self.V = self.V - jnp.sum(self.V, axis=0) / self.N

    """Physics functions"""
    def compute_ke(self):
        """Compute kinetic energy"""
        # self.KE = 0.0
        # for i in range(len(self.V)):
        #     for j in range(3):
        #         self.KE += self.V[i, j]**2 / 2.0
        # return self.KE
        self.KE = jnp.sum(self.V**2) / 2.0  # FEATURE: add mass vector
        return self.KE

    def compute_pe(self):
        """Compute potential energy"""
        self.PE = 0.0
        for i in range(self.N):
            for j in range(i):
                self.PE += potential(self.X[i], self.X[j])
        return self.PE

    def compute_temperature(self):
        if self.KE is None:
            self.KE = compute_ke()
        return self.KE / ((3 * self.N - 3) / 2.0)

    def compute_force(self):
        """Compute force between two particles"""
        for i in range(self.N):
            for j in range(i):
                # basic variables
                ri = self.X[i]
                rj = self.X[j]
                rij = ri - rj
                norm_rij = jnp.linalg.norm(rij)
                
                # PBC
                g = jnp.dot(self.inv_box, rij)
                g = g - jnp.round(g, 0)
                rij = jnp.dot(self.box, g)

                # compute pair force
                f_pair = force(potential, ri, rj) # ADD fluct-dissip terms
                
                # compute total force
                self.F_cube[i, j, :] = f_pair

        self.F = jnp.sum(self.F_cube, axis=1)
        return self.F

    """
    Minimisation
    """
    def minimise(self, n_iter=10, w=1e-7, thermo=1):
        print('step ke pe')
        
        print("Minimising... Initial PE: %.3e" % self.compute_pe())
        print("step time pe")
        tic = datetime.now()

        for i in range(1, n_iter + 1):
            self.F = self.compute_force()
            self.X += self.F * w
            self.X = self.X % np.diag(self.box)
            PE = self.compute_pe()

            toc = datetime.now()
            if i % thermo == 0:
                print(f'{i} {toc - tic} {PE}')
                
        print(f'Minimisation done. Time: {toc - tic} s')


    """
    Integration
    """
    def _euler_step(self):
        """Perform one euler step"""
        self.compute_force()
        self.V += self.F * self.dt
        self.X += self.V * self.dt

    def _verlet_step(self):
        """Perform one verlet step"""
        V2 = self.V + 0.5 * self.F * self.dt
        self.X += V2 * self.dt
        self.compute_force()
        self.V = V2 + 0.5 * self.F * self.dt

    def run(self):
        """Integrate the whole thing"""

        self.tic = datetime.now()
        # FILL
        self.toc = datetime.now()

        self._dump_observables()
        print("Done. Simulation time: %.2f s." % (self.toc - self.tic))

    def save_frames(self, it):
        """In the future, modify so that no external function is called"""
        save_xyzfile(f"Dump/dump_{it:05i}.xyz", self.bl, self.X)
        if self.dump_vel:
            save_xyzfile(f"Dump/dump_{it:05i}.vel", self.bl, self.V)

    def _dump_observables(self):
        """Save temperature, energies and pressure tensor
        components to a file"""
        df_obs = pd.DataFrame(
            {
                "temp": self.T,
                "ke": self.KE,
                "pe": self.PE,
            }
        )
        df_obs.to_csv("Dump/correl.csv")


"""
Helper functions
"""
def potential(r1, r2):
    """Compute the particle potential"""
    A, B, C = 1.0, 1.5, 1.5
    re = 2.0
    D = 1e-1
    r_cut = 6.0 # adhoc
    
    r = jnp.linalg.norm(r1 - r2)
    V = (A - B * jnp.exp(-C*(r - re)**2)) * jnp.exp(-D*r**2)
    return V if r <= r_cut else 0.0


def force(pot, r1, r2):
    """Compute force on particle 1 from particle 2"""
    return -jax.grad(pot)(r1, r2)
    

def save_xyzfile(fname, names, mat):
    """Save a coordinate matrix in a xyz format"""
    M = len(mat)
    s = f'type,x,y,z\n'
    for i in range(M):
        s += f'{names[i]},{mat[i, 0]},{mat[i, 1]},{mat[i, 2]}\n'
    open(fname, 'w').write(s)

In [ ]:
@jit(nopython=True)
def force_numba(
    X, V, rho2, bl, ip_A, ip_B, Rd, ip_N_wrap, ip_N_rho, box, gamma, kT, dt
):
    N = len(X)
    F = np.zeros((N, 3))
    Fcube = np.zeros((N, N, 3))
    inv_box = np.zeros((3, 3))
    for i in range(3):
        inv_box[i, i] = 1.0 / box[i, i]
    g = np.zeros(3)
    rij = np.zeros(3)
    vij = np.zeros(3)
    A = 0.0
    nr = 0.0
    rhoi = 0.0
    rhoj = 0.0
    nwi = 0.0
    nwj = 0.0
    nrho = 0.0
    nrm = 0.0
    fpair = 0.0

    vir = 0.0
    sigma = np.zeros(3)
    volume = np.linalg.det(box)

    for i in range(N):
        for j in range(i):
            rij = X[i] - X[j]
            g = matvecmul(inv_box, rij)
            g = g - round_numba(g)
            rij = matvecmul(box, g)
            vij = V[i] - V[j]

            nr = norm_numba(rij)
            A = ip_A[bl[i], bl[j]]
            rhoi = rho2[i, bl[j]]
            rhoj = rho2[j, bl[i]]
            rd = Rd[bl[i], bl[j]]
            nwi = ip_N_wrap[bl[i]]
            nwj = ip_N_wrap[bl[j]]
            nrho = ip_N_rho[bl[i], bl[j]]
            nrm = (nrho + 1.0) * (nrho + 2.0) * (nrho + 3.0) / (8.0 * pi * rd ** 3)

            fpair = (
                A * wr(nr)
                + ip_B
                * (rhoi ** (nwi - 1) + rhoj ** (nwj - 1))
                * nrm
                * wr(nr / rd) ** (nrho - 1)
                * nrho
                / rd
                - gamma * wr(nr) ** 2 * dot_numba(rij, vij) / nr
                + sqrt(2.0 * gamma * kT) * wr(nr) * np.random.randn() / sqrt(dt)
            )
            Fcube[i, j, :] = fpair / nr * rij
            Fcube[j, i, :] = -Fcube[i, j, :]

            vir += Fcube[i, j, :] @ rij
            sigma += Fcube[i, j, :] * rij

            if np.isnan(fpair):
                print(
                    fpair,
                    i,
                    j,
                    bl[i],
                    bl[j],
                    "\n",
                    rhoi,
                    rhoj,
                    nrm,
                    wr(nr / rd),
                    nrho,
                    gamma,
                    rij,
                    vij,
                    sqrt(dt),
                )
                assert False, "NaN in forces."

    for i in range(N):
        sigma += V[i] * V[i]

    sigma = sigma / volume
    F = np.sum(Fcube, 1)
    return F, vir, sigma


@jit(nopython=True)
def pe_numba(X, rho2, bl, ip_A, ip_B, ip_N_wrap, box):
    N = len(X)
    inv_box = np.zeros((3, 3))
    for i in range(3):
        inv_box[i, i] = 1.0 / box[i, i]
    rij = np.zeros(3)
    g = np.zeros(3)
    nr = 0.0
    pe = 0.0
    nwi = 0.0

    # standard part
    for i in range(N):
        for j in range(i):
            rij = X[i] - X[j]
            g = matvecmul(inv_box, rij)
            g = g - round_numba(g)
            nr = norm_numba(matvecmul(box, g))
            pe += ip_A[bl[i], bl[j]] * wr(nr) ** 2 / 2.0

    # many-body part
    for i in range(N):
        nwi = ip_N_wrap[bl[i]]
        pe += ip_B * np.sum(rho2[i]) ** nwi / nwi
    return pe

## 1. Testing

### 1.1. Random dev feature testing

In [3]:
# test potential and force
x1 = jnp.asarray([0., 0., 0.])
x2 = jnp.asarray([1.5, 0., 0.])
# dx = 4: push off to the left, minus
# dx = 2.5: push in to the right, plus
# dx = 1.5: push in to the left, minus

potential(x1, x2)
force(potential, x1, x2)

Array([-1.2274158,  0.       ,  0.       ], dtype=float32)

In [4]:
# random numbers
np.random.seed(42)

x1 = jnp.array(np.random.rand(3)) * 2
# x1 = np.ones(3) * 4

L = jnp.ones(3) * 2.0
box = jnp.diag(L)
inv_box = jnp.diag(1.0 / L)

In [5]:
g = np.dot(box, x1)
g

array([1.4981605, 3.8028572, 2.9279757], dtype=float32)

In [6]:
g - np.round(g, 0)

array([ 0.49816048, -0.19714284, -0.07202435], dtype=float32)

### 2. Object testing

In [16]:
np.random.rand(42)

N = 100
atom_list = [1] * N

L = 10.0
box = np.diag(np.ones(3) * L)
volume = np.prod(np.diag(box))

xyz = np.random.rand(N, 3) * L

In [8]:
sim = ReactiveFF(atom_list, xyz, box)

In [9]:
# %timeit sim.compute_ke()
sim.compute_ke()

Array(142.4362, dtype=float32)

In [10]:
# %timeit sim.compute_pe()
# sim.compute_pe()

In [11]:
sim.compute_force()

Array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.28581969e-02, -2.50381138e-02,  8.87226015e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.56067717e-02, -2.03383192e-02, -1.48625840e-02],
       [ 4.44130510e-01, -2.88624078e-01,  6.13152742e-01],
       [ 2.19781622e-01,  1.17148347e-01,  3.94941986e-01],
       [-9.79179740e-02, -2.56437119e-02, -2.98737548e-03],
       [ 1.20746620e-01, -4.64469232e-02,  3.42147090e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.31594884e-01,  1.04899384e-01,  1.97000459e-01],
       [ 3.92815262e-01, -8.33743215e-01,  7.95477688e-01],
       [-1.80593564e-03, -1.46483332e-01, -1.89177439e-01],
       [-7.79105842e-01, -9.54276323e-02, -5.44585466e-01],
       [-1.37672782e-01,  7.82314539e-02,  2.40101770e-01],
       [ 5.82082033e-01, -4.05476004e-01,  3.55965853e-01],
       [-7.29851067e-01, -9.69474316e-01

### 3. Minimisation

In [12]:
sim.minimise()

step ke pe
Minimising... Initial PE: 2.149e+02
step time pe
1 0:00:59.086865 214.87643432617188
2 0:01:49.428413 214.87646484375
3 0:02:42.409101 214.87643432617188
4 0:03:34.832728 214.87643432617188
5 0:04:24.704217 214.87643432617188
6 0:05:14.621698 214.8763885498047
7 0:06:04.242092 214.87637329101562
8 0:06:53.466507 214.8763427734375


KeyboardInterrupt: 

In [13]:
sim.minimise(20, w=1e-4)

step ke pe
Minimising... Initial PE: 2.149e+02
step time pe
1 0:00:49.044139 214.8559112548828
2 0:01:37.939084 214.8355255126953


KeyboardInterrupt: 

In [15]:
sim.minimise(20, w=5e-2)

step ke pe
Minimising... Initial PE: 2.106e+02
step time pe
1 0:00:47.400637 202.99716186523438
2 0:01:34.539080 193.6429901123047
3 0:02:21.728902 188.76344299316406


KeyboardInterrupt: 

In [18]:
lmbda = (volume / N)**(1/3)
lmbda

2.154434690031884

In [20]:
x1 = np.zeros(3)
x2 = np.zeros(3)
x2[0] = lmbda

potential(x1, x2)

Array(-0.2811924, dtype=float32)